# HMAC 기본 코드

In [1]:
import hmac
import hashlib
import base64

class HMAC:
    def __init__(self, secret_key):
        self.secret_key = secret_key.encode()

    def hmac(self, message):
        message = message.encode()
        mac = hmac.new(self.secret_key, message, hashlib.sha256)
        return base64.b64encode(mac.digest()).decode()

# 사용 예시
h = HMAC("my_secret_key")
print(h.hmac("hello"))


JjHQKvCJtdKT5vqvjpQfJ815tcvyXIuEijf/lC9F7bg=


### ✅ 2. **HMAC과 MAC의 차이**

| 항목 | MAC (Message Authentication Code) | HMAC (Hash-based MAC) |
| --- | --- | --- |
| 정의 | 메시지와 키를 조합하여 인증 코드 생성 | 해시 함수와 키를 조합하여 인증 코드 생성 |
| 구조 | 암호학적 블록 암호 또는 다른 구조 사용 가능 | SHA-256, SHA-1 등 해시 기반 알고리즘 사용 |
| 특징 | 구현 방식 다양 | 표준화되어 있고 보안 강도 입증됨 |
| 표준 | 다양한 방식 존재 | RFC 2104에 정의 |

**요약**: HMAC은 MAC의 한 종류이며, 해시 함수를 기반으로 만들어졌고 구조가 표준화되어 보안성과 구현의 일관성을 보장합니다.

---

### ✅ 3. **틀린 보기 설명**

> ❌ RFC2004에서 정의된 HMAC는 수정 없이 이용 가능한 해시함수를 사용하면 소프트웨어 구현이 어렵다
>
- **틀린 이유**: HMAC은 **기존 해시 함수 (예: SHA-256)** 를 **수정 없이 그대로 사용 가능하도록 설계**되었습니다.
- RFC 2104에서는 오히려 “해시 함수를 블랙박스처럼 쓰되, 키와 메시지를 적절히 패딩해 결합”하는 구조를 통해 **간단하고 이식성이 높고 구현이 쉬운 MAC 방식**을 제공합니다.
- 즉, 이 보기는 HMAC의 핵심 설계 목표를 반대로 서술한 오류입니다.

---

### ✅ HMAC 구조 수식 (사람이 이해하기 좋게 정리)

### 📌 기본 수식 (RFC 2104)

```
mathematica
복사편집
HMAC(K, M) = H[(K' ⊕ opad) || H((K' ⊕ ipad) || M)]

```

- `K'` : 키 `K`를 블록 크기에 맞춰 padding한 것 (SHA-256은 64바이트)
- `⊕` : XOR 연산
- `||` : 연결(concatenation)
- `ipad` : 0x36로 채운 64바이트
- `opad` : 0x5c로 채운 64바이트
- `H` : 해시 함수 (예: SHA-256)

---

### ✅ 수식이 헷갈릴 땐 파이썬 코드로 이해하세요


In [2]:
import hmac
import hashlib

key = b'secret'
message = b'hello'

h = hmac.new(key, message, hashlib.sha256)
print(h.hexdigest())

88aab3ede8d3adf94d26ab90d3bafd4a2083070c3bcce9c014ee04a443847c0b


사실 이 한 줄이 다음과 같은 구조를 따릅니다:

```
H( (K ⊕ opad) || H((K ⊕ ipad) || M) )

```

---

### ✅ 시각화 요약

```
1. 키 K가 블록 길이보다 짧으면 → 64바이트로 0 padding
2. K xor ipad → 1차 해시 입력값 구성
3. K xor opad → 2차 해시 외부 입력값 구성
4. 최종 HMAC = H(opad 패딩 + H(ipad 패딩 + message))

```

---

### ✅ 보충 설명: 왜 두 번 해시하나요?

- **ipad는 내부 키**: 메시지를 감싸서 보안성을 보장
- **opad는 외부 키**: 전체 결과를 숨기기 위한 바깥 레이어
- 두 번 해싱은 중간 해시 상태 노출 방지 및 길이 확장 공격 방지 목적

---

### ✅ HMAC 내부 구조 직접 구현 (SHA-256 기준)

In [3]:
import hashlib

BLOCK_SIZE = 64  # 블록 크기 (SHA-256은 512비트 = 64바이트)
IPAD = 0x36
OPAD = 0x5C

def xor_bytes(a: bytes, b: bytes) -> bytes:
    return bytes(x ^ y for x, y in zip(a, b))

def hmac_sha256(key: bytes, message: bytes) -> bytes:
    # 1. 키가 블록보다 크면 해시해서 줄이고
    if len(key) > BLOCK_SIZE:
        key = hashlib.sha256(key).digest()
    # 2. 키가 블록보다 짧으면 0으로 패딩
    if len(key) < BLOCK_SIZE:
        key = key.ljust(BLOCK_SIZE, b'\x00')

    # 3. ipad, opad 키 구성
    k_ipad = xor_bytes(key, bytes([IPAD] * BLOCK_SIZE))
    k_opad = xor_bytes(key, bytes([OPAD] * BLOCK_SIZE))

    # 4. 내부 해시 = H(K ⊕ ipad || message)
    inner_hash = hashlib.sha256(k_ipad + message).digest()

    # 5. 최종 해시 = H(K ⊕ opad || inner_hash)
    return hashlib.sha256(k_opad + inner_hash).digest()

# 사용 예시
key = b'secret_key'
message = b'hello'

hmac_result = hmac_sha256(key, message)
print("HMAC-SHA256:", hmac_result.hex())


HMAC-SHA256: 0f166a552b38aeb12ad07055e7bda7f8ab2f22a3a352e481de97b86f17be6bc6


### ✅ 내부 구조 요약

| 단계 | 내용 |
| --- | --- |
| 1️⃣ | 키 길이 확인 및 해시 또는 패딩 처리 |
| 2️⃣ | `ipad = 0x36`, `opad = 0x5C`로 xor |
| 3️⃣ | 내부 해시: `H(K⊕ipad |
| 4️⃣ | 외부 해시: `H(K⊕opad |
| ✅ | 결과: HMAC-SHA256 해시 출력 |

---

### 📌 참고: 표준 구현과 비교

위 코드는 아래의 간단 코드와 **동일한 결과**를 출력합니다:

In [4]:
import hmac, hashlib
h = hmac.new(b'secret_key', b'hello', hashlib.sha256)
print(h.digest())

b'\x0f\x16jU+8\xae\xb1*\xd0pU\xe7\xbd\xa7\xf8\xab/"\xa3\xa3R\xe4\x81\xde\x97\xb8o\x17\xbek\xc6'


### ✅ HMAC에서 `secret key`는 무엇인가요?

- **수신자와 송신자가 미리 공유**한 **비밀 키(secret key)**입니다.
- 대칭키 방식이라 **양쪽이 동일한 키를 갖고 있어야** HMAC을 생성하고 검증할 수 있습니다.

---

### 🔐 HMAC 사용 구조 요약

```
[ 송신자 ]
  message + secret_key → HMAC 생성 → 전송
                    ↓
[ 수신자 ]
  message + secret_key → HMAC 재계산 → 비교 (위조 여부 확인)
```

- **송신자**: 메시지와 키로 HMAC을 계산해서 함께 전송
- **수신자**: 동일한 키로 HMAC을 다시 계산 → 받은 값과 비교
- HMAC이 다르면? → 누가 내용을 바꾸었거나, 키가 틀림

---

### ❗ 비교: 디지털 서명과 차이점

| 항목 | HMAC | 디지털 서명 |
| --- | --- | --- |
| 방식 | 대칭키 | 비대칭키 |
| 키 공유 | 사전 공유 필요 | 공개키 배포 가능 |
| 용도 | 무결성 + 인증 | 무결성 + 인증 + 부인방지 |
| 사용 예 | JWT HMAC 방식, API 인증 | 전자서명, 블록체인 |